In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [170]:
# Function to get district
# param: row - <tr> HTML tag
# output: district - either string or None
def extract_district(row):
    header = row.find("th")
    if header is not None:
        anchor = header.find("a")
        if anchor is not None:
            district = anchor.contents[0]
#             dis = ''
#             for c in district:
#                 print(c)
#                 if c == '\xa0':
#                     dis.join(' ')
#                 else:
#                     dis.join(c)
#             district = str(dis)
#             district = ''.join( c for c in district if  c != '\xa0' )
#             district = ''.join( ' ' + c f)
            if ("\xa0" in district):
                district = district.split("\xa0")[0] + " " + district.split("\xa0")[1]
                
#             else:
#                 print("MAKING EXCEPTION")
#             if len(district.split("\xa0")) != 2:
#                 print("LENGHT NOT 2!")
#             print(district.split("\xa0"))
#             print(district.split(" ")[0] + " " + district.split(" ")[1])
        else:
            district = None
    else:
        district = None
    return district

In [86]:
# Function to get incumbent
# Special condition added for bug in 2018 page, will probably need to fix
# param: row - <tr> HTML tag
# output: incumbent - either string or None
def extract_incumbent(row):
    anchor = row.find_all("a")[1]
    if anchor is not None:
        incumbent = anchor.contents[0]
    else:
        incumbent = None
    if incumbent == "Bob Brady":
        incumbent = "Dwight Evans"
    return incumbent

In [87]:
# Function to get district
# param: row - <tr> HTML tag
# output: candidates - list of candidates
def extract_candidates(row):
    candidates = []
    cell = row.find_all("td")[-1]
    lst = str(cell).split("<br/>")
    for ele in lst:
        anchor = ele.split("</a>")[0].split(">")[-1]
        if len(anchor) > 5:
            candidate = anchor
        else:
            candidate = ele.split(" (")[0].split(">")[-1]
        candidates.append(candidate)
    return candidates

In [171]:
# Function to make dataframe meeting specifications for a given year
# Only based on the 2018 Wiki page so probably need to tinker
# param: year - integer of desired election year
# output: df - Pandas dataframe
def make_frame(year):
    html = "https://en.wikipedia.org/wiki/United_States_House_of_Representatives_elections,_" + str(year)
    page = requests.get(html)
    soup = BeautifulSoup(page.content, "html.parser")
    df = pd.DataFrame(columns = [["Year", "District", "Candidate", "Incumbent"]])
    tables = soup.find_all("table", class_="wikitable sortable")
#     print(len(tables))
    for table in tables[2:52]:
        rows = table.find_all("tr")
        for row in rows[2:]:
            district = extract_district(row)
            candidates = extract_candidates(row)
            incumbent = extract_incumbent(row)
            if district is not None:
                for candidate in candidates:
                    new = pd.DataFrame({"Year": [2014], "District": district, "Candidate": candidate,
                                        "Incumbent": candidate == incumbent})
                    if len(df) > 0:
                        df = df.append(new, ignore_index = True)
                    else:
                        df = new
        print(df)              
    cols_lst = df.columns
    cols = [cols_lst[3], cols_lst[1], cols_lst[0], cols_lst[2]]
    df = df[cols]
    return df

In [172]:
# Make dataframe for 2018 and write to CSV
frame_2014 = make_frame(2014)


                          Candidate   District  Incumbent  Year
0                     Bradley Byrne  Alabama 1       True  2014
1                    Burton LeFlore  Alabama 1      False  2014
2                       Martha Roby  Alabama 2       True  2014
3                      Erick Wright  Alabama 2      False  2014
4                       Mike Rogers  Alabama 3       True  2014
5                       Jesse Smith  Alabama 3      False  2014
6                   Robert Aderholt  Alabama 4       True  2014
7                         Mo Brooks  Alabama 5       True  2014
8                         Mark Bray  Alabama 5      False  2014
9                       Gary Palmer  Alabama 6      False  2014
10  Mark Lester (Democratic) 23.7%   Alabama 6      False  2014
11                       Aimee Love  Alabama 6      False  2014
12                     Terri Sewell  Alabama 7       True  2014
                          Candidate         District  Incumbent  Year
0                     Bradley Byrn

                           Candidate         District  Incumbent  Year
0                      Bradley Byrne        Alabama 1       True  2014
1                     Burton LeFlore        Alabama 1      False  2014
2                        Martha Roby        Alabama 2       True  2014
3                       Erick Wright        Alabama 2      False  2014
4                        Mike Rogers        Alabama 3       True  2014
5                        Jesse Smith        Alabama 3      False  2014
6                    Robert Aderholt        Alabama 4       True  2014
7                          Mo Brooks        Alabama 5       True  2014
8                          Mark Bray        Alabama 5      False  2014
9                        Gary Palmer        Alabama 6      False  2014
10   Mark Lester (Democratic) 23.7%         Alabama 6      False  2014
11                        Aimee Love        Alabama 6      False  2014
12                      Terri Sewell        Alabama 7       True  2014
13    

                            Candidate         District  Incumbent  Year
0                       Bradley Byrne        Alabama 1       True  2014
1                      Burton LeFlore        Alabama 1      False  2014
2                         Martha Roby        Alabama 2       True  2014
3                        Erick Wright        Alabama 2      False  2014
4                         Mike Rogers        Alabama 3       True  2014
5                         Jesse Smith        Alabama 3      False  2014
6                     Robert Aderholt        Alabama 4       True  2014
7                           Mo Brooks        Alabama 5       True  2014
8                           Mark Bray        Alabama 5      False  2014
9                         Gary Palmer        Alabama 6      False  2014
10    Mark Lester (Democratic) 23.7%         Alabama 6      False  2014
11                         Aimee Love        Alabama 6      False  2014
12                       Terri Sewell        Alabama 7       Tru

                             Candidate         District  Incumbent  Year
0                        Bradley Byrne        Alabama 1       True  2014
1                       Burton LeFlore        Alabama 1      False  2014
2                          Martha Roby        Alabama 2       True  2014
3                         Erick Wright        Alabama 2      False  2014
4                          Mike Rogers        Alabama 3       True  2014
5                          Jesse Smith        Alabama 3      False  2014
6                      Robert Aderholt        Alabama 4       True  2014
7                            Mo Brooks        Alabama 5       True  2014
8                            Mark Bray        Alabama 5      False  2014
9                          Gary Palmer        Alabama 6      False  2014
10     Mark Lester (Democratic) 23.7%         Alabama 6      False  2014
11                          Aimee Love        Alabama 6      False  2014
12                        Terri Sewell        Alaba

                                 Candidate           District  Incumbent  Year
0                            Bradley Byrne          Alabama 1       True  2014
1                           Burton LeFlore          Alabama 1      False  2014
2                              Martha Roby          Alabama 2       True  2014
3                             Erick Wright          Alabama 2      False  2014
4                              Mike Rogers          Alabama 3       True  2014
5                              Jesse Smith          Alabama 3      False  2014
6                          Robert Aderholt          Alabama 4       True  2014
7                                Mo Brooks          Alabama 5       True  2014
8                                Mark Bray          Alabama 5      False  2014
9                              Gary Palmer          Alabama 6      False  2014
10         Mark Lester (Democratic) 23.7%           Alabama 6      False  2014
11                              Aimee Love          

                              Candidate         District  Incumbent  Year
0                         Bradley Byrne        Alabama 1       True  2014
1                        Burton LeFlore        Alabama 1      False  2014
2                           Martha Roby        Alabama 2       True  2014
3                          Erick Wright        Alabama 2      False  2014
4                           Mike Rogers        Alabama 3       True  2014
5                           Jesse Smith        Alabama 3      False  2014
6                       Robert Aderholt        Alabama 4       True  2014
7                             Mo Brooks        Alabama 5       True  2014
8                             Mark Bray        Alabama 5      False  2014
9                           Gary Palmer        Alabama 6      False  2014
10      Mark Lester (Democratic) 23.7%         Alabama 6      False  2014
11                           Aimee Love        Alabama 6      False  2014
12                         Terri Sewel

                            Candidate         District  Incumbent  Year
0                       Bradley Byrne        Alabama 1       True  2014
1                      Burton LeFlore        Alabama 1      False  2014
2                         Martha Roby        Alabama 2       True  2014
3                        Erick Wright        Alabama 2      False  2014
4                         Mike Rogers        Alabama 3       True  2014
5                         Jesse Smith        Alabama 3      False  2014
6                     Robert Aderholt        Alabama 4       True  2014
7                           Mo Brooks        Alabama 5       True  2014
8                           Mark Bray        Alabama 5      False  2014
9                         Gary Palmer        Alabama 6      False  2014
10    Mark Lester (Democratic) 23.7%         Alabama 6      False  2014
11                         Aimee Love        Alabama 6      False  2014
12                       Terri Sewell        Alabama 7       Tru

In [173]:
frame_2014.to_csv("frame_2014_final.csv", index = False)